In [7]:
!python -V

Python 3.9.12


In [8]:
import pandas as pd

In [9]:
import pickle

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [12]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/08/29 12:39:34 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/29 12:39:34 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1756316531423, experiment_id='1', last_update_time=1756316531423, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [13]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
df_val = read_dataframe('./data/green_tripdata_2021-01.csv')

/tmp/ipykernel_2041/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
/tmp/ipykernel_2041/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [15]:
len(df_train), len(df_val)

(73908, 73908)

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [17]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

4.640806695934426

In [20]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [21]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-01.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [22]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)


  0%|                                                                              | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:39:44] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.04409                                                                                              
[1]	validation-rmse:10.56577                                                                                              
[2]	validation-rmse:10.12599                                                                                              
[3]	validation-rmse:9.72233                                                                                               
[4]	validation-rmse:9.35252                                                                                               
[5]	validation-rmse:9.01438                                                                                               
[6]	validation-rmse:8.70561                                                                                               
[7]	validation-rmse:8.42437                                                                                               
[8]	validation-r

[66]	validation-rmse:5.84087                                                                                              
[67]	validation-rmse:5.83940                                                                                              
[68]	validation-rmse:5.83748                                                                                              
[69]	validation-rmse:5.83558                                                                                              
[70]	validation-rmse:5.83397                                                                                              
[71]	validation-rmse:5.83254                                                                                              
[72]	validation-rmse:5.83138                                                                                              
[73]	validation-rmse:5.82985                                                                                              
[74]	validation-

[132]	validation-rmse:5.80160                                                                                             
[133]	validation-rmse:5.80129                                                                                             
[134]	validation-rmse:5.80106                                                                                             
[135]	validation-rmse:5.80077                                                                                             
[136]	validation-rmse:5.80059                                                                                             
[137]	validation-rmse:5.80034                                                                                             
[138]	validation-rmse:5.80014                                                                                             
[139]	validation-rmse:5.79989                                                                                             
[140]	validation

...

[532]	validation-rmse:4.81162                                                                             
[533]	validation-rmse:4.81145                                                                             
[534]	validation-rmse:4.81133                                                                             
[535]	validation-rmse:4.81111                                                                             
[536]	validation-rmse:4.81086                                                                             
[537]	validation-rmse:4.81055                                                                             
[538]	validation-rmse:4.81024                                                                             
[539]	validation-rmse:4.80974                                                                             
[540]	validation-rmse:4.80955                                                                             
[541]	validation-rmse:4.80942        

[608]	validation-rmse:4.78423                                                                             
[609]	validation-rmse:4.78397                                                                             
[610]	validation-rmse:4.78292                                                                             
[611]	validation-rmse:4.78270                                                                             
[612]	validation-rmse:4.78201                                                                             
[613]	validation-rmse:4.78175                                                                             
[614]	validation-rmse:4.78159                                                                             
[615]	validation-rmse:4.78141                                                                             
[616]	validation-rmse:4.78120                                                                             
[617]	validation-rmse:4.78074        

[684]	validation-rmse:4.75972                                                                             
[685]	validation-rmse:4.75960                                                                             
[686]	validation-rmse:4.75913                                                                             
[687]	validation-rmse:4.75900                                                                             
[688]	validation-rmse:4.75890                                                                             
[689]	validation-rmse:4.75874                                                                             
[690]	validation-rmse:4.75861                                                                             
[691]	validation-rmse:4.75813                                                                             
[692]	validation-rmse:4.75802                                                                             
[693]	validation-rmse:4.75789        

[760]	validation-rmse:4.74093                                                                             
[761]	validation-rmse:4.74086                                                                             
[762]	validation-rmse:4.74059                                                                             
[763]	validation-rmse:4.74045                                                                             
[764]	validation-rmse:4.74039                                                                             
[765]	validation-rmse:4.74025                                                                             
[766]	validation-rmse:4.74014                                                                             
[767]	validation-rmse:4.74003                                                                             
[768]	validation-rmse:4.73977                                                                             
[769]	validation-rmse:4.73942        

[836]	validation-rmse:4.72503                                                                             
[837]	validation-rmse:4.72490                                                                             
[838]	validation-rmse:4.72479                                                                             
[839]	validation-rmse:4.72464                                                                             
[840]	validation-rmse:4.72439                                                                             
[841]	validation-rmse:4.72417                                                                             
[842]	validation-rmse:4.72397                                                                             
[843]	validation-rmse:4.72384                                                                             
[844]	validation-rmse:4.72371                                                                             
[845]	validation-rmse:4.72356        

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)